### Import Libraries

In [1]:
from bs4 import BeautifulSoup as bs
import re
import os
import csv

In [2]:
overall_data = []

In [3]:
html_files = []
def find_html_files():
    path = "./raw html"
    ext = ".html"
    for _, _, dirfiles in os.walk(path):
        matches = (f for f in dirfiles if f.endswith(ext))
        break

    for filename in matches:
        html_files.append(filename)

In [4]:
find_html_files()
for file in html_files:
    print(file)
    with open("./raw html/"+file, encoding="utf-8") as fp:
        data = fp.read()
        soup = bs(data, 'html.parser')

        # extract post comments
        postComments = soup.select(".l9j0dhe7.ecm0bbzt.rz4wbd8a.qt6c0cv9.dati1w0a.j83agx80.btwxx1t3.lzcic4wl")

        # extract comments
        raw_comments = []
        for comment in postComments:
            comment_text = comment.select(".cxmmr5t8.oygrvhab.hcukyx3x.c1et5uql")
            x = re.findall("start;\">(.*?)<\/div>", str(comment_text))
            raw_comments.append(' '.join(x))

        cleaned_comments = []
        for comment in raw_comments:
            emojis = re.findall("img alt=\"(.*?)\"", str(comment))
            if emojis:
                if len(emojis) > 1:
                    for emoji in emojis:
                        img_tags = re.findall("<img alt.*?/>", str(comment))
                        for img_tag in img_tags:
                            comment = re.sub(img_tag, emoji, str(comment))
                else:
                    comment = re.sub("<img alt.*?/>", emojis[0], str(comment))

            soup_remove_tags = bs(comment)
            final_text = soup_remove_tags.get_text()
            cleaned_comments.append(final_text)

            date = re.match("[0-9]+", file).group(0)
            date = [date] * len(cleaned_comments)

            # list -> dict
            final_dataset = list(zip(date, cleaned_comments))
    
    overall_data.append(final_dataset)

20210316.html
20210318.html
20210315.html
20210311.html
20210317.html
20210314.html
20210312.html
20210313.html


In [5]:
with open('facebook_health_cases.csv','w', encoding='utf-8') as result_file:
    wr = csv.writer(result_file)
    #wr.writerow(['date','text'])
    for i in overall_data:
        wr.writerows(i)